In [1]:
from sklearn.datasets import make_classification


X, y = make_classification()
data = {'X': X, 'y': y}

## 1. Defining Pipelines Steps

In [ ]:
from reskit.core import Transformer
import numpy as np


def delete_first_column(data):
    data['X'] = np.delete(data['X'], 0, 1)
    return data

def mean_scaling_for_last column(data):
    mean_of_last_column = 
    data['X'][,-1] = data['X'][,-1]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


classifiers = [
        ('LR', LogisticRegression()),
        ('SVC', SVC())
]

In [4]:
from sklearn.model_selection import StratifiedKFold


grid_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
eval_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [5]:
from reskit.core import Pipeliner


steps = [
        ('Scaler', scalers),
        ('Classifier', classifiers)
]

## 2. Defining Grid Search Parameters

In [6]:


param_grid = {
        'LR' : {
                'penalty' : ['l1', 'l2']
        },
        'SVC' : {
                'kernel' : ['linear', 'poly', 'rbf', 'sigmoid']
        }
}

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid)
pipe.plan_table

,Scaler,Classifier
0,minmax,LR
1,minmax,SVC
2,standard,LR
3,standard,SVC


## 3. Launching Experiment

In [7]:
pipe.get_results(data=data, scoring=['roc_auc'])

Removed previous results file -- results.csv.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4


,Scaler,Classifier,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,minmax,LR,0.976,0.0205913,{'penalty': 'l1'},0.982,0.016,[ 0.96 0.97 1. 1. 0.98]
1,minmax,SVC,0.984,0.0162481,{'kernel': 'rbf'},0.97,0.0167332,[ 0.96 0.95 0.97 1. 0.97]
2,standard,LR,0.986,0.0174356,{'penalty': 'l1'},0.984,0.0224499,[ 0.99 0.94 1. 1. 0.99]
3,standard,SVC,0.968,0.0193907,{'kernel': 'sigmoid'},0.966,0.0195959,[ 0.97 0.96 0.96 1. 0.94]


transform_with_caching

In [ ]:
pipe.transform_with_caching

get_grid_search_results

get_scores

get_results